In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader

import os
import numpy as np


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


Using device: cuda:0


In [3]:
data_directory = "./Data"

# Load the data from the numpy files
x_train_all = np.load(os.path.join(data_directory, "x_train_all.npy"),mmap_mode='r')
x_val_all = np.load(os.path.join(data_directory, "x_val_all.npy"),mmap_mode='r')


y_train_all = np.load(os.path.join(data_directory, "y_train_all.npy"))
y_val_all = np.load(os.path.join(data_directory, "y_val_all.npy"))


#calculate the mean and the standard deviation of the data
data_mean = np.mean(x_train_all)
data_std = np.std(x_train_all)

#normalize the data using the above values
x_train_all_normalized = (x_train_all - data_mean) / data_std
x_val_all_normalized = (x_val_all - data_mean) / data_std


# Convert the numpy arrays to PyTorch tensors
x_train_all_tensor = torch.tensor(x_train_all_normalized, dtype=torch.float32)
x_val_all_tensor = torch.tensor(x_val_all_normalized, dtype=torch.float32)


y_train_all_tensor = torch.tensor(y_train_all, dtype=torch.int16)
y_val_all_tensor = torch.tensor(y_val_all, dtype=torch.int16)


# Print the shapes to verify
print("x_train_all shape:", x_train_all_tensor.shape)
print("x_val_all shape:", x_val_all_tensor.shape)

print("y_train_all shape:", y_train_all_tensor.shape)
print("y_val_all shape:", y_val_all_tensor.shape)




x_train_all shape: torch.Size([891, 128, 128, 128, 1])
x_val_all shape: torch.Size([128, 128, 128, 128, 1])
y_train_all shape: torch.Size([891])
y_val_all shape: torch.Size([128])


In [4]:
batch_size = 8

train_dataset = TensorDataset(x_train_all_tensor, y_train_all_tensor)
val_dataset = TensorDataset(x_val_all_tensor, y_val_all_tensor)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


# Import necessary libraries
import torch
from torch.utils.data import DataLoader

# Assuming you have already created your train_loader
# ...

# Iterate through the train_loader to get a batch of data
for inputs, labels in train_loader:
    # Print the shape of the batch
    print("Inputs shape:", inputs.shape)
    print("Labels shape:", labels.shape)
    break

Inputs shape: torch.Size([8, 128, 128, 128, 1])
Labels shape: torch.Size([8])


In [5]:
# Loop through a few batches from the train_loader
for batch_idx, (inputs, labels) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}")
    
    # Print the shapes of the loaded data and labels
    print("Inputs shape:", inputs.shape)
    print("Labels shape:", labels.shape)
    
    # Print some sample values from the inputs and labels
    print("Sample inputs:", inputs[0])  # Print the first sample's input
    print("Sample labels:", labels[0])  # Print the first sample's label
    
    if batch_idx == 2:  # Stop after a few batches
        break

Batch 1
Inputs shape: torch.Size([8, 128, 128, 128, 1])
Labels shape: torch.Size([8])
Sample inputs: tensor([[[[-0.2967],
          [-0.2967],
          [-0.2967],
          ...,
          [-0.2967],
          [-0.2967],
          [-0.2967]],

         [[-0.2967],
          [-0.2967],
          [-0.2967],
          ...,
          [-0.2967],
          [-0.2967],
          [-0.2967]],

         [[-0.2967],
          [-0.2967],
          [-0.2967],
          ...,
          [-0.2967],
          [-0.2967],
          [-0.2967]],

         ...,

         [[-0.2967],
          [-0.2967],
          [-0.2967],
          ...,
          [-0.2967],
          [-0.2967],
          [-0.2967]],

         [[-0.2967],
          [-0.2967],
          [-0.2967],
          ...,
          [-0.2967],
          [-0.2967],
          [-0.2967]],

         [[-0.2967],
          [-0.2967],
          [-0.2967],
          ...,
          [-0.2967],
          [-0.2967],
          [-0.2967]]],


        [[[-0.2967],
   

In [6]:
import torch.nn as nn

class Simple3DCNNModel(nn.Module):
    def __init__(self):
        super(Simple3DCNNModel, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d(kernel_size=2)
        self.fc = nn.Linear(16 * 64 * 64 * 64, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Create an instance of the simplified model
simple_model = Simple3DCNNModel()


In [7]:
# Create an instance of your model
model = Simple3DCNNModel()

In [10]:
learning_rate = 0.001
# Define your loss function
criterion = nn.CrossEntropyLoss()

# Define your optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode

    for inputs, labels in train_loader:
    
        inputs = inputs.permute(0, 4, 1, 2, 3)
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels.long())   # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the model's parameters
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Batch Loss: {loss.item():.4f}")
    # Validation after each epoch (optional)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            inputs = inputs.permute(0, 4, 1, 2, 3)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels.long()).item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Training Loss: {loss.item():.4f}, '
              f'Validation Loss: {val_loss/len(val_loader):.4f}, '
              f'Validation Accuracy: {(100 * correct / total):.2f}%')

Epoch [1/3], Batch Loss: 2.5672
Epoch [1/3], Batch Loss: 56.1406
Epoch [1/3], Batch Loss: 38.2995
Epoch [1/3], Batch Loss: 121.0733
Epoch [1/3], Batch Loss: 62.3385
Epoch [1/3], Batch Loss: 35.8351
Epoch [1/3], Batch Loss: 0.0000
Epoch [1/3], Batch Loss: 348.0807
Epoch [1/3], Batch Loss: 204.1739
Epoch [1/3], Batch Loss: 4.0246
Epoch [1/3], Batch Loss: 4.1743
Epoch [1/3], Batch Loss: 82.0968
Epoch [1/3], Batch Loss: 24.0044
Epoch [1/3], Batch Loss: 36.4521
Epoch [1/3], Batch Loss: 67.0299
Epoch [1/3], Batch Loss: 10.3608
Epoch [1/3], Batch Loss: 38.0186
Epoch [1/3], Batch Loss: 15.3614
Epoch [1/3], Batch Loss: 15.6969
Epoch [1/3], Batch Loss: 36.9133
Epoch [1/3], Batch Loss: 35.4311
Epoch [1/3], Batch Loss: 14.9845
Epoch [1/3], Batch Loss: 22.4039
Epoch [1/3], Batch Loss: 34.3968
Epoch [1/3], Batch Loss: 18.1010
Epoch [1/3], Batch Loss: 35.4103
Epoch [1/3], Batch Loss: 11.1775
Epoch [1/3], Batch Loss: 2.0387
Epoch [1/3], Batch Loss: 28.8849
Epoch [1/3], Batch Loss: 13.4425
Epoch [1/3],

KeyboardInterrupt: 